In [1]:
import pandas as pd
from itertools import product

In [2]:
import inspect, re

def varname(p):
  for line in inspect.getframeinfo(inspect.currentframe().f_back)[3]:
    m = re.search(r'\bvarname\s*\(\s*([A-Za-z_][A-Za-z0-9_]*)\s*\)', line)
    if m:
      return m.group(1)

In [3]:
df = pd.read_excel('TestList.xlsx')
segments = sorted(list(set(df.Segment.to_list())))
segments

['ChunkStorage',
 'ChunkingAlgorithm',
 'Embedder',
 'FAISS Vector',
 'Query',
 'Reranker',
 'TextNormalize']

### ChunkStorage

In [4]:
scoring_function = ['BM25F']
K1 = [1, 1.2, 1.5, 2.0]
B = [0.5, 0.75, 1, 1.25]

WhooshChunkStorage = [
    {
        'scoring_function': sf,
        'K1': k1,
        'B': b
    }
    for sf, k1, b in product(scoring_function, K1, B)
]
ChunkStorage = [{
    'method': varname(WhooshChunkStorage),
    'params': i
    } for i in WhooshChunkStorage]

### ChunkingAlgorithm

Word count

In [5]:
word_amount = [128, 256, 512, 1024, 2048]
overlay = [0.1, 0.25, 0.5]

WordCountingChunkingAlgorithm = [
    {
        'word_amount': wa,
        'overlay': o
    }
    for wa, o in product(word_amount, overlay)
]

Sentence count

In [6]:
sentence_amount = [2, 4, 8, 16, 32]
overlay = [0.1, 0.25, 0.5]

SentenceChunkingAlgorithm = [
    {
        'sentence_amount': sa,
        'overlay': o
    }
    for sa, o in product(sentence_amount, overlay)
]

In [7]:
ChunkingAlgorithm = [{
    'method': varname(WordCountingChunkingAlgorithm),
    'params': i
    } for i in WordCountingChunkingAlgorithm] + [{
        
    'method': varname(SentenceChunkingAlgorithm),
    'params': i
    } for i in SentenceChunkingAlgorithm]


### Embedder

In [8]:
model = ['BAAI/LLM-Embedder', 'SFR-Embedding-Mistral', 'mxbai-embed-large-v1', 'Alibaba-NLP/gte-large-en-v1.5']

HuggingFaceEmbedder = [{'model': i} for i in model]

In [9]:
Embedder = [{
    'method': 'BoWEmbedder',
    'params': None}] +[{
        
    'method': 'TfidfEmbedder',
    'params': None}] + [{
        
    'method': varname(HuggingFaceEmbedder),
    'params': i
    } for i in HuggingFaceEmbedder]
            

### FAISS Vector

In [10]:
FAISS_Vector = [{
    'method': 'FaissIndexFlatL2',
    'params': None}] +[{
        
    'method': 'FaissIndexFlatIP',
    'params': None}]

### LLM

In [11]:
model = ['7b', '8b', '12b']
LLM = [{'model': i} for i in model]
LLM = [{
    'method': varname(LLM),
    'params': i
    } for i in LLM]

### Create final tets file

In [12]:
len([{
    # varname(ChunkStorage): i[0],
    varname(ChunkingAlgorithm): i[0],
    varname(Embedder): i[1],
    varname(FAISS_Vector): i[2],
    varname(LLM): i[3]
}
for i in product( ChunkingAlgorithm, Embedder, FAISS_Vector, LLM)])

1080